In [115]:
#imports go here
import pandas as pd
import numpy as np
import os
import datetime
import time
import math

In [375]:
DIR_PATH = '../../Volumes/KevSSDrive/NBA/data/csv/sportvu/'
EVENT_PATH = '../../Volumes/KevSSDrive/NBA/data/csv/events/'
allFiles = os.listdir('../../Volumes/KevSSDrive/NBA/data/csv/merged')

In [376]:
def insert_row(df, new_row, r):
    df = df.shift()
    #df.iloc[range(r+1, len(df)+1)] = df.iloc[range(r, len(df))]
    df.iloc[r] = new_row
    return df

def func(group):
    return group.head(1)

In [ ]:
aadef rowCounts_chull(gameid):
    #if you want to do all files in a directory, use a loop
    #for csv in allFiles:
        #gameid = csv[7:17]
        #load game data
    for i in range(1):
        all_movements = pd.read_csv((DIR_PATH+gameid+'.csv'))
        if len(all_movements) > 500:
            print "all_movements is good"
        else:
            print "all_movements is not good"
        pbp = pd.read_csv((EVENT_PATH+gameid+'.csv'))
        if len(pbp) > 10:
            print "pbp is good"
        else:
            print "pbp is bad"
        df_total = pd.DataFrame()
        #df_game is just the ball, and only while it's in the air
        df_game = pd.DataFrame(all_movements, copy=True)
        df_game = df_game[df_game['player_id'] == -1]
        print "len of df_game (ball) = {}".format(len(df_game))
        df_game = df_game[df_game['radius'] > 8]
        df_game.reset_index(drop=True, inplace=True)
        print "len of df_game (ball in air) = {}".format(len(df_game))
        #initialize threeDist and threedistz to 0
        df_game['threedist'] = 0.0
        df_game['threedistz'] = 0.0
        for row in range(len(df_game)):
            x_loc = df_game.x_loc.iloc[row]
            if x_loc < 47:
                #XY distance to the basket
                df_game.set_value(row, 'threedist', (math.sqrt(math.pow((x_loc - 5.25), 2) + 
                                                               math.pow((df_game['y_loc'].iloc[row] - 25), 2))))
                                 
                #XYZ distance to the basket
                df_game.set_value(row,'threedistz', (math.sqrt(math.pow((x_loc - 5.25), 2) + 
                                                               math.pow((df_game['y_loc'].iloc[row] - 25), 2) + 
                                                               math.pow((df_game.radius.iloc[row] - 10), 2))))
            else:
                #XY distance to the basket
                df_game.set_value(row, 'threedist', (math.sqrt(math.pow((x_loc - 88.75), 2) + 
                                                               math.pow((df_game['y_loc'].iloc[row] - 25), 2))))
                #XYZ distance to the basket
                df_game.set_value(row, 'threedistz', (math.sqrt(math.pow((x_loc - 88.75), 2) + 
                                                                math.pow((df_game['y_loc'].iloc[row] - 25), 2) + 
                                                                math.pow((df_game.radius.iloc[row] - 10), 2))))
        df_game.sort_values(by=['quarter', 'game_clock'], ascending=[True, False], inplace=True)
        df_game.drop_duplicates(subset=['game_clock'], inplace=True)
        df_game.reset_index(drop=True, inplace=True)
        #df_game.to_csv('df_game.csv',index=False)
            #Find the start and end of plays when ball is in the air
            #print df_game.head()
        shot_break_end = pd.DataFrame(df_game, copy=True)
            #initialize lead_game_clock to 0
        shot_break_end['lead_game_clock'] = 0
        shot_break_end['lead_game_clock'] = shot_break_end['game_clock'].shift(periods=-1)
        shot_break_end = shot_break_end[(shot_break_end['game_clock'] - shot_break_end['lead_game_clock'] > 1)]            
        shot_break_end.drop_duplicates(subset=['game_clock', 'quarter'], inplace=True)
        shot_break_end.rename(columns={"game_clock":'game_clock_end'}, inplace=True)
        shot_break_end = shot_break_end[['game_clock_end', 'quarter']]
            
        shot_break_start = pd.DataFrame(df_game, copy=True)
        shot_break_start['lag_game_clock'] = 0
        shot_break_start['lag_game_clock'] = shot_break_start['game_clock'].shift(periods=1)
        shot_break_start = shot_break_start[(shot_break_start['lag_game_clock'] - shot_break_start['game_clock'] > 1)]
        shot_break_start.drop_duplicates(subset=['game_clock', 'quarter'], inplace=True)
        shot_break_start.rename(columns={'game_clock':'game_clock_start'}, inplace=True)
        shot_break_start = shot_break_start[['game_clock_start', 'quarter']]
            
        #Create dataframe with start and end times of ball in the air
        r = 0
        new_row = [df_game['game_clock'].iloc[0], df_game['quarter'].iloc[0]] #Start with first time
        length = len(shot_break_start)
        shot_row = shot_break_start.iloc[length-1]
        shot_break_start = insert_row(shot_break_start, new_row, r)
        shot_break_start = shot_break_start.append(shot_row, ignore_index=True) #Add the last time
        shot_break = pd.concat([shot_break_start.reset_index(drop=True), shot_break_end.drop('quarter', axis=1).reset_index(drop=True)], axis=1)
        #shot_break = shot_break_start.merge(shot_break_end, on='quarter', how='inner')
            
        
        #Now that we have the start/end times, lets start by filtering out our dataset to these times
        #Also, lets get rid of any players that are less than 22 feet
        #Assign a new id to these plays - shot_id
        sumtotal = pd.DataFrame()
        for i in range(len(shot_break)):
            df_event = pd.DataFrame(df_game, copy=True)
            #print "len df_game = {}".format(len(df_game))
            #print "len of df_event (equal to df_game) = {}".format(len(df_event))
            df_event = df_event[df_event['quarter'] == shot_break['quarter'].iloc[i]]
            #print "len of df_event (first quarter only) = {}".format(len(df_event))
            df_event = df_event[df_event['game_clock'] <= shot_break['game_clock_start'].iloc[i]]
            df_event.reset_index(drop=True, inplace=True)
            #print "len of df_event (game_clock <= game_clock_start) = {}".format(len(df_event))
            df_event = df_event[df_event['game_clock'] > shot_break['game_clock_end'].iloc[i]]
            #df_event.reset_index(drop=True, inplace=True)
            #print "len of df_event (final condition) loop {} = {}".format(i, len(df_event))
            #df_event = df_event[(df_event['threedist'].max() - df_event['threedist'].min()) > 22]
            #print "max = {}, min = {}".format(df_event['threedist'].max(), df_event['threedist'].min())
            if not ((df_event['threedist'].max() - df_event['threedist'].min()) > 22):
                continue
            df_event['shot_id'] = i
            sumtotal = sumtotal.append(df_event, ignore_index=True)
            #print sumtotal.head()
        #sumtotal.to_csv('sumtotal.csv',index=False)
        #This gives us a dataFrame of the ball in the air, on plays, where it goes greater than 22 feet
            
        #The next step is matching this data to the play by play data:
            
        #This brings in all the 3 point shots in the play by play data
        # this is one way to bring additional information in
        pbp_shot = pd.DataFrame(pbp, copy=True)
        pbp_shot = pbp_shot[['EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'HOMEDESCRIPTION',
                           'VISITORDESCRIPTION', 'PCTIMESTRING', 'PERIOD', 'PLAYER1_ID']]
        pbp_shot['threepoint'] = 0
        #pbp_shot['threepoint'] = pbp_shot.loc[(pbp_shot['VISITORDESCRIPTION'].str.contains('3PT') | pbp_shot['HOMEDESCRIPTION'].str.contains('3PT'))]
        visitor_three = pd.DataFrame(pbp_shot, copy=True)
        visitor_three.dropna(subset=['VISITORDESCRIPTION'], inplace=True)
        visitor_three.reset_index(inplace=True, drop=True)
        for n in range(len(visitor_three)):
            if '3PT' in visitor_three['VISITORDESCRIPTION'].iloc[n]:
                visitor_three.set_value(n, 'threepoint', 1)
        home_three = pd.DataFrame(pbp_shot, copy=True)
        home_three.dropna(subset=['HOMEDESCRIPTION'], inplace=True)
        home_three.reset_index(inplace=True, drop=True)
        for m in range(len(home_three)):
            if '3PT' in home_three['HOMEDESCRIPTION'].iloc[m]:
                home_three.set_value(m, 'threepoint', 1)
        pbp_shot = visitor_three.append(home_three, ignore_index=True)
        pbp_shot.to_csv('merged_pbp.csv',index=False)
        pbp_shot.reset_index(inplace=True, drop=True)
        #print pbp_shot.head()
        pbp_shot.sort_values(by='EVENTNUM', ascending=True, inplace=True)
        pbp_shot = pbp_shot.loc[pbp_shot['threepoint'] == 1]
        pbp_shot.reset_index(drop=True, inplace=True)
        print "len(pbp_shot) = {}".format(len(pbp_shot))
        print pbp_shot.info()
        for row in range(len(pbp_shot)):
            x = time.strptime(pbp_shot.loc[row, 'PCTIMESTRING'], '%M:%S')
            pbp_shot.loc[row, 'game_clock'] = datetime.timedelta(minutes=x.tm_min, seconds=x.tm_sec).total_seconds()
        pbp_shot.to_csv('pbp_shot.csv', index=False)    
        sumtotal3 = pd.DataFrame()
        for q in range(1, 5):
            df_merge = pd.DataFrame(sumtotal, copy=True)
            #print sumtotal.head()
            df_merge = df_merge[df_merge['quarter'] == q]
            if len(df_merge) > 0:
                events = pd.DataFrame(df_merge, copy=True)
                events = events.drop_duplicates(subset='shot_id')
                pbp_q = pd.DataFrame(pbp_shot, copy=True)
                pbp_q = pbp_q.loc[pbp_q.PERIOD == q]
                for i in range(1, len(events)):
                    df_merge2 = pd.DataFrame(df_merge, copy=True)
                    df_merge2 = df_merge2[df_merge2['shot_id'] == events['shot_id'].iloc[i]]
                    merge_time = pd.DataFrame(df_merge2, copy=True)
                    merge_time = merge_time['game_clock'].max()
                    timeb = abs(pbp_q['game_clock'] - merge_time) < 5
                    #indexc = 0
                    timeb.reset_index(drop=True, inplace=True)
                    #print "Timeb = {}".format(timeb)
                    for indexc in range(len(timeb)):
                        if timeb[indexc] == 1:
                            break
                    if timeb.sum() > 0:
                        df_merge2['EVENTNUM'] = pbp_q['EVENTNUM'].iloc[indexc]
                        df_merge2['EVENTMSGTYPE'] = pbp_q['EVENTMSGTYPE'].iloc[indexc]
                        df_merge2['PLAYER1_ID'] = pbp_q['PLAYER1_ID'].iloc[indexc]
                    else:
                        df_merge2['EVENTNUM'] = 999 #999 indicates no match found
                        df_merge2['EVENTMSGTYPE'] = 999
                        df_merge2['PLAYER1_ID'] = 999
                    sumtotal3 = sumtotal3.append(df_merge2, ignore_index=True)
        sumtotal3 = sumtotal3[sumtotal3['EVENTMSGTYPE'] != 999] #Remove any no match plays
            
        #Now we have a dataframe of 3 point plays from when the ball leaves the shooters hand to 
        #when it reaches the basket
            
        #Finds the point where the ball leaves the shooters hand
        sumtotal3.to_csv('beforeGroups.csv',index=False)
        grouping = pd.DataFrame(sumtotal3, copy=True)
        grouping = grouping.groupby(['shot_id'])
        df_startshot = grouping.apply(func)
        df_startshot = df_startshot[['shot_id', 'EVENTMSGTYPE', 'game_clock', 'quarter', 'PLAYER1_ID', 'shot_clock']]
        df_startshot.sort_values(by=['quarter', 'game_clock'], ascending=[True, False], inplace=True)
        df_startshot.to_csv('afterGroups.csv',index=False)
            
        #Loops through each three point play
        for i in range(len(df_startshot)):
                
            #Get start of the play
            df_startplay = pd.DataFrame(all_movements, copy=True)
            df_startplay = df_startplay[df_startplay['quarter'] ==
                                            df_startshot.quarter.iloc[i]]
            df_startplay = df_startplay[df_startplay['game_clock'] >=
                                           df_startshot.game_clock.iloc[i]]
            df_startplay = df_startplay[df_startplay['player_id'] == -1]
            df_startplay.drop_duplicates(subset=['quarter', 'game_clock'], inplace=True)
            df_startplay.sort_values(by=['quarter', 'game_clock'], inplace=True)
            df_startplay.dropna(subset=['shot_clock'], how='any', inplace=True)
            df_startplay['lead_shot_clock'] = 0
            df_startplay['lead_shot_clock'] = df_startplay['shot_clock'].shift(periods=-1)
            df_startplay = df_startplay[(df_startplay['shot_clock'] - df_startplay['lead_shot_clock']) > 1]
            df_startplay = df_startplay.head(1)
            df_startplay.reset_index(inplace=True, drop=True)
            
            #Get the ball/player data now that we have the start/end time
            if len(df_startplay) > 0:
                df_play = pd.DataFrame(all_movements, copy=True)
                df_play = df_play[df_play['quarter'] == df_startshot.quarter.iloc[i]]
                
                df_play = df_play[df_play['game_clock'] <= df_startplay['game_clock'].iloc[0]]
                df_play = df_play[df_play['game_clock'] >= df_startshot.game_clock.iloc[i]]
                df_play['playid'] = i
                df_play.drop_duplicates(subset=['player_id', 'quarter', 'game_clock'], inplace=True)
                df_play.sort_values(by=['game_clock', 'player_id'], ascending=[False, True], inplace=True)
                    
                #Rotate plays depending upon location of the shot
                
                if df_play.tail(1)['x_loc'].iloc[0] > 47:
                    df_play['x_loc'] = 94 - df_play.x_loc
                    df_play['y_loc'] = 50 - df_play.y_loc
                df_play['gameid'] = gameid
                #Adding some of the pbp data
                df_play['EVENTMSGTYPE'] = df_startshot['EVENTMSGTYPE'].iloc[i]
                    
                df_play['PLAYER1_ID'] = df_startshot['PLAYER1_ID'].iloc[i]
                df_total = df_total.append(df_play, ignore_index=True)
    return df_total

In [522]:
final = rowCounts_chull('0021500001')

all_movements is good
pbp is good
len of df_game (ball) = 222551
len of df_game (ball in air) = 27567
len(pbp_shot) = 56
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 9 columns):
EVENTNUM              56 non-null int64
EVENTMSGTYPE          56 non-null int64
EVENTMSGACTIONTYPE    56 non-null int64
HOMEDESCRIPTION       27 non-null object
VISITORDESCRIPTION    29 non-null object
PCTIMESTRING          56 non-null object
PERIOD                56 non-null int64
PLAYER1_ID            56 non-null int64
threepoint            56 non-null int64
dtypes: int64(6), object(3)
memory usage: 4.0+ KB
None


In [523]:
final.to_csv('Python_df_total_final.csv',index=False)

In [445]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35496 entries, 0 to 35495
Data columns (total 14 columns):
team_id         35496 non-null int64
player_id       35496 non-null int64
x_loc           35496 non-null float64
y_loc           35496 non-null float64
radius          35496 non-null float64
game_clock      35496 non-null float64
shot_clock      35496 non-null float64
quarter         35496 non-null int64
game_id         35496 non-null int64
event_id        35496 non-null int64
playid          35496 non-null int64
gameid          35496 non-null object
EVENTMSGTYPE    35496 non-null int64
PLAYER1_ID      35496 non-null int64
dtypes: float64(5), int64(8), object(1)
memory usage: 3.8+ MB


In [410]:
m = [False, False, True, True, False, True, False]

In [79]:
df = pd.read_csv(DIR_PATH+'merged_0021500001.csv',low_memory=False)
#df1 = pd.read_csv(EVENT_PATH+'0021500001.csv')

In [63]:
df = df.groupby(by='event_id')

In [395]:
[10, 5, 6] & [5]

TypeError: unsupported operand type(s) for &: 'list' and 'list'

In [69]:
df.head(1)

,team_id,player_id,x_loc,y_loc,radius,game_clock,shot_clock,quarter,game_id,event_id,EVENTMSGTYPE,EVENTMSGACTIONTYPE,SCORE
0,-1,-1,47.25031,26.14806,6.76567,720.00,24.00,1,21500001,1,10.0,0.0,NaN
1650,-1,-1,66.19088,20.37273,1.94592,717.41,21.40,1,21500001,2,2.0,42.0,NaN
7183,-1,-1,66.19088,20.37273,1.94592,717.41,21.40,1,21500001,3,4.0,0.0,NaN
12716,-1,-1,9.64407,28.11885,3.90356,702.12,6.48,1,21500001,4,5.0,45.0,NaN
14124,-1,-1,59.72093,16.17623,4.84552,697.00,24.00,1,21500001,5,1.0,80.0,2 - 0
20962,-1,-1,8.56875,18.68857,12.91258,683.22,9.41,1,21500001,6,1.0,80.0,2 - 2
28387,-1,-1,84.39988,29.36863,12.85015,662.29,7.40,1,21500001,7,1.0,101.0,4 - 2
34162,-1,-1,6.76334,26.24917,12.02962,647.34,13.30,1,21500001,8,2.0,1.0,NaN
41587,-1,-1,6.76334,26.24917,12.02962,647.34,13.30,1,21500001,9,4.0,0.0,NaN
49012,-1,-1,69.83522,15.69193,5.00253,626.41,24.00,1,21500001,10,2.0,58.0,NaN


In [59]:
df1.groupby(by')

Index([u'GAME_ID', u'EVENTNUM', u'EVENTMSGTYPE', u'EVENTMSGACTIONTYPE',
       u'PERIOD', u'WCTIMESTRING', u'PCTIMESTRING', u'HOMEDESCRIPTION',
       u'NEUTRALDESCRIPTION', u'VISITORDESCRIPTION', u'SCORE', u'SCOREMARGIN',
       u'PERSON1TYPE', u'PLAYER1_ID', u'PLAYER1_NAME', u'PLAYER1_TEAM_ID',
       u'PLAYER1_TEAM_CITY', u'PLAYER1_TEAM_NICKNAME',
       u'PLAYER1_TEAM_ABBREVIATION', u'PERSON2TYPE', u'PLAYER2_ID',
       u'PLAYER2_NAME', u'PLAYER2_TEAM_ID', u'PLAYER2_TEAM_CITY',
       u'PLAYER2_TEAM_NICKNAME', u'PLAYER2_TEAM_ABBREVIATION', u'PERSON3TYPE',
       u'PLAYER3_ID', u'PLAYER3_NAME', u'PLAYER3_TEAM_ID',
       u'PLAYER3_TEAM_CITY', u'PLAYER3_TEAM_NICKNAME',
       u'PLAYER3_TEAM_ABBREVIATION'],
      dtype='object')

In [36]:
df['radius_shift'] = df['radius'].shift(periods=-1)

In [48]:
df['x_loc'].min()

-8.1988599999999998

In [285]:
df['player_id'].iloc[len(df)-1]

203083

In [78]:
df.loc[df.PERIOD == 1]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,threepoint
0,21500001,0,12,0,1,8:12 PM,12:00,NaN,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
1,21500001,1,10,0,1,8:12 PM,12:00,Jump Ball Horford vs. Drummond: Tip to Ilyasova,NaN,NaN,...,Pistons,DET,5,101141,Ersan Ilyasova,1.610613e+09,Detroit,Pistons,DET,False
2,21500001,2,2,42,1,8:13 PM,11:41,Horford BLOCK (1 BLK),NaN,MISS Drummond 2' Driving Layup,...,NaN,NaN,4,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL,False
3,21500001,3,4,0,1,8:13 PM,11:39,Bazemore REBOUND (Off:0 Def:1),NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
4,21500001,4,5,45,1,8:13 PM,11:37,Bazemore Out of Bounds - Bad Pass Turnover Tur...,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
5,21500001,5,1,80,1,8:13 PM,11:21,NaN,NaN,Marc Morris 13' Step Back Jump Shot (2 PTS) (D...,...,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN,False
6,21500001,6,1,80,1,8:13 PM,11:00,Millsap 12' Step Back Jump Shot (2 PTS) (Korve...,NaN,NaN,...,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN,False
7,21500001,7,1,101,1,8:14 PM,10:44,NaN,NaN,Caldwell-Pope 9' Driving Floating Jump Shot (2...,...,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN,False
8,21500001,8,2,1,1,8:14 PM,10:27,MISS Horford 20' Jump Shot,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
9,21500001,9,4,0,1,8:14 PM,10:25,NaN,NaN,Jackson REBOUND (Off:0 Def:1),...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False


In [70]:
df['threepoint'] = (df.loc[:,'HOMEDESCRIPTION'].str.contains('3PT') | df.loc[:, 'VISITORDESCRIPTION'].str.contains('3PT'))

In [73]:
df.loc[85, 'threepoint'] == 1

True

In [72]:
df.head(100)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,threepoint
0,21500001,0,12,0,1,8:12 PM,12:00,NaN,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
1,21500001,1,10,0,1,8:12 PM,12:00,Jump Ball Horford vs. Drummond: Tip to Ilyasova,NaN,NaN,...,Pistons,DET,5,101141,Ersan Ilyasova,1.610613e+09,Detroit,Pistons,DET,False
2,21500001,2,2,42,1,8:13 PM,11:41,Horford BLOCK (1 BLK),NaN,MISS Drummond 2' Driving Layup,...,NaN,NaN,4,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL,False
3,21500001,3,4,0,1,8:13 PM,11:39,Bazemore REBOUND (Off:0 Def:1),NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
4,21500001,4,5,45,1,8:13 PM,11:37,Bazemore Out of Bounds - Bad Pass Turnover Tur...,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
5,21500001,5,1,80,1,8:13 PM,11:21,NaN,NaN,Marc Morris 13' Step Back Jump Shot (2 PTS) (D...,...,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN,False
6,21500001,6,1,80,1,8:13 PM,11:00,Millsap 12' Step Back Jump Shot (2 PTS) (Korve...,NaN,NaN,...,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN,False
7,21500001,7,1,101,1,8:14 PM,10:44,NaN,NaN,Caldwell-Pope 9' Driving Floating Jump Shot (2...,...,Pistons,DET,0,0,NaN,NaN,NaN,NaN,NaN,False
8,21500001,8,2,1,1,8:14 PM,10:27,MISS Horford 20' Jump Shot,NaN,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False
9,21500001,9,4,0,1,8:14 PM,10:25,NaN,NaN,Jackson REBOUND (Off:0 Def:1),...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False


In [56]:
len(df)

2451093

In [83]:
x = time.strptime(df.loc[5, 'PCTIMESTRING'], '%M:%S')

In [86]:
datetime.timedelta(minutes=x.tm_min, seconds=x.tm_sec).total_seconds()

681.0

In [87]:
df.loc[5, 'PCTIMESTRING']

'11:21'

In [95]:
row = 5
df.loc[row, 'game_clock'] = datetime.timedelta(minutes=x.tm_min, seconds=x.tm_sec).minutes

AttributeError: 'datetime.timedelta' object has no attribute 'minutes'

In [96]:
df.head(10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,threepoint,game_clock
0,21500001,0,12,0,1,8:12 PM,12:00,NaN,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,720
1,21500001,1,10,0,1,8:12 PM,12:00,Jump Ball Horford vs. Drummond: Tip to Ilyasova,NaN,NaN,...,DET,5,101141,Ersan Ilyasova,1.610613e+09,Detroit,Pistons,DET,False,720
2,21500001,2,2,42,1,8:13 PM,11:41,Horford BLOCK (1 BLK),NaN,MISS Drummond 2' Driving Layup,...,NaN,4,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL,False,701
3,21500001,3,4,0,1,8:13 PM,11:39,Bazemore REBOUND (Off:0 Def:1),NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,699
4,21500001,4,5,45,1,8:13 PM,11:37,Bazemore Out of Bounds - Bad Pass Turnover Tur...,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,697
5,21500001,5,1,80,1,8:13 PM,11:21,NaN,NaN,Marc Morris 13' Step Back Jump Shot (2 PTS) (D...,...,DET,0,0,NaN,NaN,NaN,NaN,NaN,False,0:12:00
6,21500001,6,1,80,1,8:13 PM,11:00,Millsap 12' Step Back Jump Shot (2 PTS) (Korve...,NaN,NaN,...,ATL,0,0,NaN,NaN,NaN,NaN,NaN,False,660
7,21500001,7,1,101,1,8:14 PM,10:44,NaN,NaN,Caldwell-Pope 9' Driving Floating Jump Shot (2...,...,DET,0,0,NaN,NaN,NaN,NaN,NaN,False,644
8,21500001,8,2,1,1,8:14 PM,10:27,MISS Horford 20' Jump Shot,NaN,NaN,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,627
9,21500001,9,4,0,1,8:14 PM,10:25,NaN,NaN,Jackson REBOUND (Off:0 Def:1),...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,625
